
# De waarde van voetbaltransfers

In deze notebook ga ik analyseren wat de waarde is van transfers in de Nederlandse Eredivisie voetbal.
Elk jaar worden er grote bedragen uitgegeven door clubs. Ik ben benieuwd wat voor effect dit heeft op de uiteindelijke stand op de ranglijst.

Voor deze analyse richt ik me op de aankopen van de klassieke 'grote 3', namelijk Ajax, Feyenoord en PSV. De datasets die ik ga gebruiken zijn de bijgeleverde bestanden: `transfers.csv` en `standen.csv`. Deze bestanden heb ik gemaakt door de website www.transfermarkt.nl te scrapen. Voor de code van de scraper verwijs ik je naar het meegeleverde bestand `scraper.py`. 
De datasets betreffen de inkomende transfers van alle eredivisieclubs en de ranglijsten van de seizoenen 2000/2001 tot en met 2020/2021. Dit is inclusief spelers die uit de jeugd zijn overgekomen en spelers die transfervrij zijn overgekomen. 

## Wat kost een punt in de eindstand in Euros als  je naar de totale transfersom in een jaar kijkt?

Aan de hand van de totale aankoopsom per jaar per club, ga ik berekenen wat deze transfersom aan punten heeft opgeleverd. 

Verder ga ik een kijken wat de gevolgen zijn van het aandeel van de jeugdspelers in dit puntenaantal: loont het om jeugdspelers kansen te geven of kun je beter in ervaren spelers investeren?

Heeft het zin om op koopjesjacht te gaan en vooral transfervrije spelers te halen of ik goedkoop ook hier duurkoop?

Wat is het effect van de gemiddelde leeftijd van de aankopen op de eindstand als we de jeugdspelers buiten beschouwing laten?

Op bovenstaande vragen hoop ik een antwoord te vinden door gebruik van bovenstaande datasets en ik wil dit visualiseren door gebruik te maken van verschillende grafieken.



### Imports

Ik zal al mijn imports voor dit notebook in een blok zetten. Dat is het blok hieronder.


In [1]:
import pandas as pd
# import numpy as np


### De basis-dataframes

Als eerst ga ik de csv-bestanden importeren en in een dataframe zetten. Dit zal resulteren in 2 dataframes:
1. standen
2. transfers

Deze dataframes zullen de basis zijn voor mijn analyses.
Ik zal de 'head' van de dataframes tonen.



#### Standen

Import van `standen.csv` en verwerking in een dataframe.
Ik maak een lijst aan voor de kolomnamen.

In [2]:
stand_namen = ['jaar', 'rangschikking', 'club', 'wedstrijden', 'gewonnen', 'gelijk', 'verloren', 'voor/tegen', 'doelsaldo', 'punten']
standen = pd.read_csv('standen.csv', names=stand_namen)
#FC Twente en NAC Breda zitten er dubbel in vanwege toevoeging '*' Dit pas ik hieronder aan
standen['club'].replace(['FC Twente *', 'NAC Breda *'], ['FC Twente', 'NAC Breda'], inplace=True)
standen.head()

,jaar,rangschikking,club,wedstrijden,gewonnen,gelijk,verloren,voor/tegen,doelsaldo,punten
0,2000,1,PSV,34,25,8,1,73:23,50,83
1,2000,2,Feyenoord,34,21,3,10,67:37,30,66
2,2000,3,Ajax,34,18,7,9,85:43,42,61
3,2000,4,Roda JC,34,17,8,9,59:41,18,59
4,2000,5,FC Utrecht,34,17,8,9,58:43,15,59



#### Transfers

Import van `transfers.csv` en verwerking in een dataframe. Ook hier maak ik een lijst aan voor de kolommen.


In [3]:
transfer_namen = ['jaar', 'club', 'speler', 'vorige club', 'leeftijd', 'positie', 'transfersom']
transfers = pd.read_csv('transfers.csv', names=transfer_namen)


In de dataframes worden net iets afwijkende clubnamen gebruikt.
Hieronder zorg ik ervoor dat de clubnamen in beide dataframes overeenkomen.


In [4]:
standen_clubs = standen.club.unique()
transfer_clubs = transfers.club.unique()

foute_namen = []

for club in standen_clubs:
    if club not in transfer_clubs:
        foute_namen.append(club)
        
print(foute_namen)

meer_foute_namen = []

for club in transfer_clubs:
    if club not in standen_clubs:
        meer_foute_namen.append(club)

print(meer_foute_namen)

# transfers['club'].replace(transfer_clubs, standen_clubs, inplace=True)
# transfers.club.unique()

['Ajax', 'Vitesse', 'Heerenveen', 'NEC', 'Sparta', 'Excelsior']
['AFC Ajax', 'Vitesse Arnhem', 'Sparta Rotterdam', 'NEC Nijmegen', 'sc Heerenveen', 'Excelsior Rotterdam']



Nu ga ik de afwijkende namen matchen.


In [10]:
transfers['club'].replace(['AFC Ajax', 'Vitesse Arnhem', 'Sparta Rotterdam', 'NEC Nijmegen', 'sc Heerenveen', 'Excelsior Rotterdam'],
                         ['Ajax', 'Vitesse', 'Sparta', 'NEC', 'Heerenveen', 'Excelsior'],
                         inplace=True)

transfers.club.unique()

array(['Fortuna Sittard', 'RBC Roosendaal', 'AZ Alkmaar', 'De Graafschap',
       'Ajax', 'Vitesse', 'Sparta', 'NEC', 'Willem II', 'NAC Breda',
       'PSV', 'FC Twente', 'Heerenveen', 'RKC Waalwijk', 'Feyenoord',
       'FC Groningen', 'FC Utrecht', 'Roda JC', 'FC Den Bosch',
       'FC Zwolle', 'Excelsior', 'FC Volendam', 'ADO Den Haag',
       'Heracles Almelo', 'VVV-Venlo', 'PEC Zwolle', 'SC Cambuur',
       'Go Ahead Eagles', 'FC Dordrecht', 'FC Emmen'], dtype=object)


## Punten per EURO transfersom

Om te berekenen wat de totale transfersom per jaar is per club en daarna te berekenen wat elk punt heeft 'gekost' ga ik eerst uit de tabel `transfers` de koopsom per jaar bepalen en deze toevoegen als een kolom in de standen tabel.
Vervolgens ga ik nog een kolom toevoegen waarin de transfersom gedeeld wordt door het behaalde aantal punten: De kosten per punt.


In [11]:
# Berekenen van de totale transfersom per jaar per club
totale_som = transfers.groupby(['jaar', 'club']).transfersom.sum()
totale_som

jaar  club         
2000  AZ Alkmaar       3300000.0
      Ajax             5300000.0
      De Graafschap          0.0
      FC Groningen      450000.0
      FC Twente         675000.0
                         ...    
2020  RKC Waalwijk           0.0
      Sparta            100000.0
      VVV-Venlo         360000.0
      Vitesse                0.0
      Willem II        1075000.0
Name: transfersom, Length: 378, dtype: float64


Om de transfersommen per jaar en per club toe te voegen aan het `standen dataframe` maak ik een lijst met de transfersommen en deze voeg ik aan het dataframe toe.


In [8]:
list = []
# loop door dataframe om een lijst aan te maken in dezelfde volgorde als het dataframe 'standen'
for index, row in standen.iterrows():
    list.append(totale_som[row['jaar'], row['club']])

# Voeg lijst toe aan aan dataframe
standen['transfersom'] = list



Nu kan ik eindelijk de kosten per punt berekenen. Ook deze kolom voeg ik toe aan het dataframe `standen`.


In [13]:
# maak een kolom aan met de standen per punt
standen['kosten per punt'] = round(standen['transfersom'] / standen['punten'], 2)
# filter de top 3 uit het dataframe
topclub_standen = standen[(standen['club'] == 'Ajax') | (standen['club'] == 'Feyenoord') | (standen['club'] == 'PSV')]
topclub_standen

,jaar,rangschikking,club,wedstrijden,gewonnen,gelijk,verloren,voor/tegen,doelsaldo,punten,transfersom,kosten per punt
0,2000,1,PSV,34,25,8,1,73:23,50,83,14000000.0,168674.70
1,2000,2,Feyenoord,34,21,3,10,67:37,30,66,4700000.0,71212.12
2,2000,3,Ajax,34,18,7,9,85:43,42,61,5300000.0,86885.25
18,2001,1,Ajax,34,22,7,5,73:34,39,73,15500000.0,212328.77
19,2001,2,PSV,34,20,8,6,77:32,45,68,17700000.0,260294.12
...,...,...,...,...,...,...,...,...,...,...,...,...
344,2019,3,Feyenoord,25,14,8,3,50:35,15,50,11300000.0,226000.00
345,2019,4,PSV,26,14,7,5,54:28,26,49,31500000.0,642857.14
360,2020,1,Ajax,34,28,4,2,102:23,79,88,63200000.0,718181.82
361,2020,2,PSV,34,21,9,4,74:35,39,72,15500000.0,215277.78
